In [2]:
import torch
import numpy as np
import scipy

x= [[1,0,0], [1,1,0], [1,0,1], [0,1,1]]
p = [[0.8, 0.2], [0.1,0.9], [0.7,0.3], [0.4,0.6]]
tensor_x = torch.Tensor(x)
tensor_p = torch.Tensor(p)

/home/shkim/anaconda3/envs/torch_1.13/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
x_coo = tensor_x.to_sparse_coo()

In [4]:
x_coo.indices()

tensor([[0, 1, 1, 2, 2, 3, 3],
        [0, 0, 1, 0, 2, 1, 2]])

In [5]:
srt = x_coo.indices()[:,x_coo.indices().sort(1)[1][1]]
print(srt)
xx = torch.nested.nested_tensor([srt[:,0:3], srt[:,3:5], srt[:,5:6]])

tensor([[0, 1, 2, 1, 3, 2, 3],
        [0, 0, 0, 1, 1, 2, 2]])


/home/shkim/anaconda3/envs/torch_1.13/lib/python3.10/site-packages/torch/nested/__init__.py:47: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642969563/work/aten/src/ATen/NestedTensorImpl.cpp:175.)
  nt = torch._nested_tensor_from_tensor_list(new_data, dtype, None, device, pin_memory)


In [6]:
x_coo.indices().sort(1)[1][1]

tensor([0, 1, 3, 2, 5, 4, 6])

In [7]:
col_change_loc = torch.argwhere(torch.diff(srt[1]))[:,0] + 1
_len = srt.shape[1] * torch.ones(1)
_zero = torch.zeros(1)
_diff_ = torch.cat([_zero, col_change_loc, _len])
_split = tuple(torch.diff(_diff_).to(torch.int).tolist())

dnd = list(torch.split(srt, _split, dim=1))
dfd = torch.nested.nested_tensor(dnd) # 1.13 torch.nested.nested_tensor(...)
xfx = dfd.to_padded_tensor(-1)

###################################################

print(xfx)
# extended = torch.cat((tensor_p, torch.zeros((1,2))), 0)
# xdx = extended[xfx[:,0,:]]
# print(xdx)
# 1 - (1 - xdx).prod(1)

tensor([[[ 0,  1,  2],
         [ 0,  0,  0]],

        [[ 1,  3, -1],
         [ 1,  1, -1]],

        [[ 2,  3, -1],
         [ 2,  2, -1]]])


In [8]:
ind = x_coo.indices()
sp_sorted = tensor_p[ind[0]].T[:,x_coo.indices().sort(1)[1][1]]
print(sp_sorted)

tensor([[0.8000, 0.1000, 0.7000, 0.1000, 0.4000, 0.7000, 0.4000],
        [0.2000, 0.9000, 0.3000, 0.9000, 0.6000, 0.3000, 0.6000]])


In [13]:
x_coo = torch.tensor([[1,0,0,1], [1,1,0,0], [1,0,1,1], [0,1,1,0]]).to_sparse_coo()
srt = x_coo.indices()[:,x_coo.indices().sort(1)[1][1]]

e_num = x_coo.shape[1]
p_num = 2

col_change_loc = torch.argwhere(torch.diff(srt[1]))[:,0] + 1
_len = srt.shape[1] * torch.ones(1)
_zero = torch.zeros(1)
_diff_ = torch.cat([_zero, col_change_loc, _len])
_split = torch.diff(_diff_).to(torch.int)

print(srt)
dnd = torch.split(srt[0], _split.tolist())
dfd = torch.nested.nested_tensor(list(dnd))
xfx = dfd.to_padded_tensor(x_coo.shape[0])
###################################################
# extended = torch.cat((tensor_p, torch.zeros((1,2))), 0)
extended = torch.nn.functional.pad(tensor_p, (0,0,0,1), 'constant', 0)
print(xfx.view(-1))
xdx = extended[xfx]
xdx = extended.index_select(dim=0, index=xfx.view(-1)).view((e_num, -1 ,p_num))
print(xdx)

part_prob = 1 -  (1-xdx).prod(1)
print(part_prob)

max_prob = xdx.max(dim=1)
print(max_prob.values)


vol_S = xdx.sum(dim=1).sum(dim=0)
vol_V = vol_S.sum()
vol_Sinv = vol_V - vol_S

ww = (vol_V/(vol_S * vol_Sinv)) * (1 - torch.eye(tensor_p.shape[1]))
# ww = (1 - torch.eye(tensor_p.shape[1]))
cut_S = torch.trace(torch.linalg.multi_dot([part_prob, ww, part_prob.T]))

zz = ww @ part_prob.T
print(zz)

print(part_prob.T.mul(zz).sum())

print(cut_S)

tensor([[0, 1, 2, 1, 3, 2, 3, 0, 2],
        [0, 0, 0, 1, 1, 2, 2, 3, 3]])
tensor([0, 1, 2, 1, 3, 4, 2, 3, 4, 0, 2, 4])
tensor([[[0.8000, 0.2000],
         [0.1000, 0.9000],
         [0.7000, 0.3000]],

        [[0.1000, 0.9000],
         [0.4000, 0.6000],
         [0.0000, 0.0000]],

        [[0.7000, 0.3000],
         [0.4000, 0.6000],
         [0.0000, 0.0000]],

        [[0.8000, 0.2000],
         [0.7000, 0.3000],
         [0.0000, 0.0000]]])
tensor([[0.9460, 0.9440],
        [0.4600, 0.9600],
        [0.8200, 0.7200],
        [0.9400, 0.4400]])
tensor([[0.8000, 0.9000],
        [0.4000, 0.9000],
        [0.7000, 0.6000],
        [0.8000, 0.3000]])
tensor([[0.4204, 0.4275, 0.3206, 0.1959],
        [0.4213, 0.2048, 0.3652, 0.4186]])
tensor(2.0829)
tensor(2.0829)


In [ ]:
partition = torch.argmax(tensor_p, dim=1)

_idx = x_coo.indices()
_val = x_coo.values()

x_scipy = scipy.sparse.coo_matrix((_val, _idx), shape=x_coo.shape)
deg = np.multiply(x_scipy.sum(1), np.eye(4, dtype=int))
print(x_scipy.dot(x_scipy.T).todense() - deg)

[[0 1 2 0]
 [1 0 1 1]
 [2 1 0 1]
 [0 1 1 0]]


In [ ]:
feats = torch.FloatTensor([[1,1,0,1],[0,1,0,0],[0,0,0,0.5]])
_adj = torch.FloatTensor(([0,1,1],[1,0,0],[1,0,0])).to_sparse_coo()
_adj.matmul(feats)

tensor([[0.0000, 1.0000, 0.0000, 0.5000],
        [1.0000, 1.0000, 0.0000, 1.0000],
        [1.0000, 1.0000, 0.0000, 1.0000]])

In [ ]:
x_coo = torch.tensor([[1,0,0], [1,1,0], [1,0,1], [0,1,1]]).to_sparse_coo()

ind = x_coo.indices()
srt = x_coo.indices()[:,x_coo.indices().sort(1)[1][1]]
row_change_loc = torch.argwhere(torch.diff(ind[0]))[:,0] + 1
col_change_loc = torch.argwhere(torch.diff(srt[1]))[:,0] + 1
_len = srt.shape[1] * torch.ones(1, dtype=int)
_zero = torch.zeros(1, dtype=int)
_diff_r = torch.cat([_zero, row_change_loc, _len])
_diff_c = torch.cat([_zero, col_change_loc, _len])
_split_r = torch.diff(_diff_r).to(torch.int).tolist()
_split_c = torch.diff(_diff_c).to(torch.int).tolist()

drd = torch.split(ind[1] ,_split_r)
dcd = torch.split(srt[0], _split_c)

In [ ]:
x_coo = torch.tensor([[1,0,0], [1,1,0], [1,0,1], [0,1,1]])
print(x_coo.to_dense())
print(torch.mm(x_coo, x_coo.T).to_dense())

tensor([[1, 0, 0],
        [1, 1, 0],
        [1, 0, 1],
        [0, 1, 1]])
tensor([[1, 1, 1, 0],
        [1, 2, 1, 1],
        [1, 1, 2, 1],
        [0, 1, 1, 2]])


In [ ]:
with open('../dataset/hgr_benchmark_set/ISPD98_ibm01.hgr.part.4', 'r') as f:
    part = torch.tensor([int(x.strip()) for x in f.readlines()])

xx = part.unique(return_counts = True)[1]
print(xx)
print(xx.sum())

tensor([3039, 4480, 3100, 2133])
tensor(12752)


In [ ]:
x = [[9,0.9,0.1],[9,0.9,0.1],[9,0.9,0.1],[2,1,7]]
t_x = torch.tensor(x)/10

xx = t_x.index_select(dim=0, index=torch.tensor([0,1,2]))
(1-(1-xx).prod(0)).sum()

tensor(1.2751)

In [17]:
__inc = torch.tensor([
    [1, 0, 0, 0, 0],
    [1, 1, 0, 0, 0],
    [1, 0, 0, 1, 0],
    [0, 1, 1, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1],
    [0, 0, 0, 0, 1],
], dtype=float).to_sparse_coo()

A = __inc.matmul(__inc.T)
print(A.to_dense())
A2 = A.matmul(A)
print(A2.to_dense())

neighs = A2.indices()

print(__inc.index_select(dim=0, index=neighs[0,torch.argwhere(neighs[1] == 5).flatten()]))

tensor([[1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 2., 1., 1., 0., 0., 0., 0.],
        [1., 1., 2., 0., 0., 1., 0., 0.],
        [0., 1., 0., 2., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 2., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 1., 1., 1.]], dtype=torch.float64)
tensor([[3., 4., 4., 1., 0., 1., 0., 0.],
        [4., 7., 5., 4., 1., 1., 0., 0.],
        [4., 5., 7., 1., 0., 4., 1., 1.],
        [1., 4., 1., 6., 3., 0., 0., 0.],
        [0., 1., 0., 3., 2., 0., 0., 0.],
        [1., 1., 4., 0., 0., 7., 4., 4.],
        [0., 0., 1., 0., 0., 4., 3., 3.],
        [0., 0., 1., 0., 0., 4., 3., 3.]], dtype=torch.float64)
tensor(indices=tensor([[0, 1, 1, 2, 2, 3, 3, 4, 5],
                       [0, 0, 1, 0, 3, 3, 4, 4, 4]]),
       values=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1.]),
       size=(6, 5), nnz=9, dtype=torch.float64, layout=torch.sparse_coo)
